# Using SVM 

In [15]:
import pandas as pd
import numpy as np

import os
import sys
sys.path.append(os.path.abspath('../src'))

import string
import nltk
from nltk import ngrams

# Caching stopwords
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('english'))

from nltk.stem.porter import PorterStemmer

from fact_classification import *

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC


In [16]:
df, df_crowdsourced, df_ground_truth = data_loading()

## Testing with just bigrams based on the text

In [17]:
df['Text_stemmed'] = stem(df)

In [18]:
df_train, df_test = test_train_split(df)

In [19]:
train_tfid, test_tfid, vocab  =  tfid(train = df_train.Text, test = df_test.Text, n_gram_range = 2)

In [37]:
method = SVC(class_weight = 'balanced', gamma = 0.3, random_state = 42) #setting gamma to avoid overfitting
pred_train, pred_test = predict_it(train_tfid, df_train.Verdict, test_tfid, method = method) # predicting with basemodel

In [38]:
df_score_train = score_it(df_train.Verdict, pred_train, features = 'tfid', algorithm = 'SVC')
df_score_test = score_it(df_test.Verdict, pred_test, features = 'tfid', algorithm = 'SVC')

In [39]:
df_score_train

,alogrithm,features,p_NFS,p_UFS,p_CFS,p_wavg,r_NFS,r_UFS,r_CFS,r_wavg,f_NFS,f_UFS,f_CFS,f_wavg
0,SVC,tfid,0.990376,0.882861,0.824311,0.940856,0.90985,0.99783,0.969467,0.932636,0.948407,0.936831,0.891016,0.933888


In [40]:
df_score_test

,alogrithm,features,p_NFS,p_UFS,p_CFS,p_wavg,r_NFS,r_UFS,r_CFS,r_wavg,f_NFS,f_UFS,f_CFS,f_wavg
0,SVC,tfid,0.779026,0.385081,0.540831,0.669928,0.815932,0.306581,0.529453,0.680589,0.797052,0.341376,0.535082,0.674461


The results are considerably better than the baseline model

## Testing with POS tagging on Text